Random Forest Algorithm

## Bootstraping

> We first split the features in to different datasets, Then we create Decision trees for eachof the dataset

> Next we introduce the data to all the trees and get the prediction


> Next we get the majority voting

## Aggregation

> Combining results from multiple models

---
Combination of two of above task called bagging

why we use bootstraping and feature selection?

We use bootstraping for randomize and prevent overfitting.
Other than like a single Decision tree the random forest has multiple trees so that it is suit for numeric datasets

> Importing nessasary Librries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

> Reading the dataset(After EDA)

Dividing the training sets and test sets

In [3]:
X_train = pd.read_csv("./Training_Set.csv")
X_test = pd.read_csv("./Testing_Set .csv")
y_train = pd.read_csv("./y_train.csv")
y_test = pd.read_csv("./y_test.csv")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(644, 19) (162, 19)
(644, 1) (162, 1)


Importing the RandomForest Model

In [4]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier

In [5]:
RandomForest = RandomForestClassifier()

> This is the random forest values without Tuning and using the basic model of scikit learn

In [6]:
acc_score = []
roc_score = []
f1 = []
name_model = []


# Training the Model
RandomForest.fit(X_train,y_train)

#Setting the Prediction value form the RandomForest
y_pred =  RandomForest.predict(X_test)

#Testing the accuracy of the Model
acc_score.append(accuracy_score(y_test, y_pred))

#Testing the ROC
roc_score.append(roc_auc_score(y_test, y_pred))

#Tesing the F1 score
f1.append(f1_score(y_test, y_pred, average='weighted'))

name_model.append(type(RandomForest).__name__)

result = pd.DataFrame(
    {'Model Name' : name_model,
     'accuracy': acc_score,
     'roc auc' : roc_score,
     'f1-weighted' : f1}
)
result.sort_values('f1-weighted',ascending=False)

<ipython-input-6-de6923e68b0b>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RandomForest.fit(X_train,y_train)


,Model Name,accuracy,roc auc,f1-weighted
0,RandomForestClassifier,0.833333,0.83196,0.832982


In [7]:
from joblib import dump, load
dump(RandomForest, 'random_forest_model.joblib')


['random_forest_model.joblib']

Hyperparameter Tuning using the Grid Search

> Setting the Parameter grid

> This is used to set what typical values we can use for tuning

In [8]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'log2'],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced'],
    'criterion': ['gini', 'entropy']
}

> This is the implementation of grid search. In this we can check model performance for various parameter values without tuning manually.

In [9]:
# Initialize the RandomForestClassifier
rf = RandomForestClassifier()

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           scoring=['accuracy', 'roc_auc', 'f1'],
                           refit='roc_auc',  # Refitting based on F1-weighted score
                           cv=5,  # 5-fold cross-validation
                           verbose=5, n_jobs=-1)



### Checking values

In [10]:
# Fit the grid search to your data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, None],
                         'max_features': ['auto', 'log2'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             refit='roc_auc', scoring=['accuracy', 'roc_auc', 'f1'], verbose=5)

> for getting the best estimater model parameters

In [11]:
print(grid_search.best_estimator_)


RandomForestClassifier(criterion='entropy', max_depth=20, max_features='log2',
                       min_samples_leaf=2)


In [12]:
print(grid_search.best_params_)

{'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}


In [13]:
print(grid_search.best_score_)

0.9324522285452694


### For further checking of the values that grid search has obtained

In [14]:
df2 = pd.DataFrame(grid_search.cv_results_)
df2 = df2.sort_values("rank_test_roc_auc")
df2.to_csv("cv_results.csv")

In [20]:
RandomForestModel = RandomForestClassifier(criterion='entropy', max_depth=20, max_features='log2',
                       min_samples_leaf=2)

In [16]:
from joblib import dump, load
dump(RandomForestModel, 'random_forest_model.joblib')

['random_forest_model.joblib']

## This is the values gained after Tuning and Cross validation

In [21]:
name_model = []
name_model.append(type(RandomForestModel).__name__)
# Assuming you have already performed GridSearchCV and have the scores
acc_score_cv = grid_search.cv_results_['mean_test_accuracy']
roc_auc_score_cv = grid_search.cv_results_['mean_test_roc_auc']
f1_score_cv = grid_search.cv_results_['mean_test_f1']

# Create a DataFrame to display the cross-validated results
cv_results_df = pd.DataFrame({
    'mean_test_accuracy': acc_score_cv,
    'mean_test_roc_auc': roc_auc_score_cv,
    'mean_test_f1': f1_score_cv
})
Final_result_cv = cv_results_df.sort_values('mean_test_roc_auc',ascending=False).reset_index(drop=True).head(1)
Final_result_cv.insert(0, 'Model Name', name_model)
Final_result_cv

,Model Name,mean_test_accuracy,mean_test_roc_auc,mean_test_f1
0,RandomForestClassifier,0.857195,0.932452,0.866809


### Saving the model for demonstration

In [18]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
from sklearn.model_selection import train_test_split

# Assuming X_train, X_test, y_train, y_test are defined

# Initialize the lists to store scores and model name
acc_score = []
roc_score = []
f1 = []
name_model = []

# Initialize the RandomForestClassifier
RandomForestModel = RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_features='log2', min_samples_leaf=2,
                       min_samples_split=5)

# Training the model
RandomForestModel.fit(X_train, y_train)

# Save the fitted model to a file


# Setting the Prediction value form the RandomForest
y_pred = RandomForestModel.predict(X_test)

# Testing the accuracy of the Model
acc_score.append(accuracy_score(y_test, y_pred))

# Testing the ROC
roc_score.append(roc_auc_score(y_test, y_pred))

# Testing the F1 score
f1.append(f1_score(y_test, y_pred))  # added 'average' parameter to match GridSearchCV scoring

# Adding model name
name_model.append(type(RandomForestModel).__name__)

# Create a DataFrame to store the results
result = pd.DataFrame({
    'Model Name': name_model,
    'accuracy': acc_score,
    'roc auc': roc_score,
    'f1 score': f1
})





<ipython-input-18-81e0c94de98d>:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RandomForestModel.fit(X_train, y_train)


In [ ]:
dump(RandomForestModel,'random_forest_model.joblib')